# Creando un modelo de clasificación de texto

Objetivos de la práctica:
- Conocer cómo crear un algoritmo de clasificación de texto usando la librería transformers.
- Conocer el entorno de HuggingFace ([datasets](https://huggingface.co/datasets), [models](https://huggingface.co/models), [spaces](https://huggingface.co/spaces),...)

Este notebook está basado en el [curso de HuggingFace](https://huggingface.co/course/chapter3/1?fw=pt).

Para este notebook es conveniente que compruebes que la opción de GPU está activada (Runtime -> Change Runtime Type).

## Creando una cuenta en HuggingFace

Lo primero que debemos hacer es [crear una cuenta de HuggingFace](https://huggingface.co/join). Además deberás crear un [token de escritura](https://huggingface.co/docs/hub/security-tokens). Estos dos pasos solo los deberás hacer la primera vez.

## Instalando librerías

Por defecto, el entorno de Google Colab no tiene instaladas las librerías de [HuggingFace](https://huggingface.co/), por lo que vamos a hacer en primer lugar es instalar las librerías: [Transformers](https://huggingface.co/docs/transformers/index), [Datasets](https://huggingface.co/docs/datasets/index), y [Evaluate](https://huggingface.co/docs/evaluate/index).

In [1]:
!pip install datasets evaluate transformers[sentencepiece] accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00


A continuación nos conectamos al hub de huggingface, lo que nos permitirá subir nuestros modelos a este entorno. Al ejecutar la siguiente celda aparecerá un widget en el cual tendremos que copiar el token generado en el primer paso y pulsar en el botón login.

In [1]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset

Para este ejemplo vamos a utilizar el [dataset de muchocine](https://huggingface.co/datasets/muchocine) que contiene reseñas de películas en español. Para cada una de las reseñas se incluye una valoración entre 1 y 5. Nuestro objetivo es crear un modelo para automatizar la valoración de una película a partir de su reseña.

Comenzamos descargando el dataset.

In [2]:
from datasets import load_dataset
raw_dataset = load_dataset("muchocine")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for muchocine contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/muchocine
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/3872 [00:00<?, ? examples/s]

Veamos el contenido de este dataset.

In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_summary', 'star_rating'],
        num_rows: 3872
    })
})

Podemos ver que tenemos un objeto DatasetDict que puede verse como un diccionario. Dicho diccionario contiene un atributo `train`. En algunos casos veremos que el dataset ya está divido en conjuntos de entrenamiento y test, pero en este caso no es así, por lo que lo tendremos que dividirlo nosotros. Pero antes de esto vamos a ver alguna de las frases del dataset, para lo que tenemos que acceder al atributo `train`.

In [4]:
raw_dataset['train']

Dataset({
    features: ['review_body', 'review_summary', 'star_rating'],
    num_rows: 3872
})

Con el anterior comando vemos que tenemos un Dataset con tres columnas: `review_body`, `review_summary`, y `star_rating`. Si queremos ver el contenido del dataset, lo podemos transformar a formato pandas y verlo como una tabla.  

In [5]:
raw_dataset['train'].to_pandas()

,review_body,review_summary,star_rating
0,"""May, ¿Quieres ser mi amigo?"" es una de esas p...","May, ¿quieres ser mi amigo?",3
1,Es todo un alivio que ante tanta película que ...,Cómo ponerse en la piel de un kamikaze,3
2,"Una fiesta llena de excesos, rubias despampana...","Silicona, esteroides, pactos demoníacos y otra...",0
3,"Zoom nos cuenta la historia de Jack Shepard, a...",Una comedia entretenida y poca cosa más para v...,1
4,Luc Besson dirige esta película basada en sus ...,"Luc Besson sabe manejar la acción, y aquí lo d...",3
...,...,...,...
3867,Hoy voy de rollo cultural y os diré que esta p...,"Película dirigida a un público adolescente, co...",2
3868,"El día que me dispuse a ver ""El Manantial de l...",Bergman en estado puro. Duro a la par que sens...,3
3869,"No es que no me guste, pero reconozco que siem...",Por fin vemos un Almodóvar que retrata escenas...,3
3870,"Tony Scott es, en mi opinión, un correcto dire...",Dos horas entretenidillas de thriller con cier...,1


Para poder entrenar un modelo con este dataset es necesario tokenizarlo. Cada modelo tokeniza de una manera distinta, por lo que es necesario indicar el modelo para tokenizar el texto. En nuestro caso vamos a utilizar un modelo llamado [Electricidad](https://huggingface.co/mrm8488/electricidad-base-discriminator).

In [6]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_checkpoint = "mrm8488/electricidad-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

Definimos una función para tokenizar el texto. Notar que para otros datasets
será necesario cambiar el valor de "review_summary" por la columna que queramos
tokenizar, el resto del código no hará falta tocarlo.

In [7]:
def tokenize_function(example):
    return tokenizer(example["review_summary"], truncation=True)

Tokenizamos el dataset y lo mostramos.

In [9]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/3872 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_summary', 'star_rating', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3872
    })
})

Podemos ver que han aparecido tres nuevas columnas ('input_ids', 'token_type_ids' y 'attention_mask') que serán utilizadas para entrenar el modelo.

Para poder entrenar un modelo de clasificación de texto, es necesario que nuestro dataset tenga una columna llamada `label`, por lo que tenemos que renombrar nuestra columna `star_rating`.

In [10]:
tokenized_dataset = tokenized_dataset.rename_column('star_rating','label')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_summary', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3872
    })
})

Además, necesitamos partir nuestro dataset en un conjunto de entrenamiento y en un conjunto de test. Para lo cual, vamos a:
1. Revolver el dataset.
2. Calcular el número de elementos de nuestro dataset.
3. Dividir el dataset en dos trozos (80% para entrenar y 20% para testear).
4. Construir un nuevo dataset con un conjunto de entrenamiento y uno de test.

Notar que este paso es necesario porque el dataset no está dividido previamente en entrenamiento y test, si ese fuera el caso, este paso no sería necesario.

In [11]:
from datasets import DatasetDict,Dataset
# 1. Revolvemos el dataset con el método shuffle
new_tokenized_dataset = tokenized_dataset["train"].shuffle()
# 2. Calculamos el número de elementos del dataset
len_dataset = len(tokenized_dataset["train"])
# 3. Partimos el dataset en dos trozos
train_dataset = tokenized_dataset["train"][0:int(len_dataset*0.8)]
test_dataset = tokenized_dataset["train"][int(len_dataset*0.8):]
new_dataset = DatasetDict({"train":Dataset.from_dict(train_dataset),"test":Dataset.from_dict(test_dataset)})


Por último, antes de definir nuestro modelo tenemos que definir una función que se va a encargar de preparar los datos para que sean procesados de manera eficiente por el modelo.

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Modelo

Pasamos ahora a definir el modelo, lo primero que vamos a definir son los argumentos con los que vamos a entrenar nuestro modelo. Aunque podemos configurar el entrenamiento de múltiples maneras, en este caso vamos a utilizar los valores por defecto, y solo vamos a modificar el nombre con el que se va a guardar nuestro modelo, que en este caso va a ser `clasificador-muchocine`. Además le vamos a pedir que nos muestre cómo de bien funciona el modelo a medida que se va entrenando mediante la `evaluation_strategy` con valor `epoch`.

In [13]:
from transformers import TrainingArguments
training_args = TrainingArguments("clasificador-muchocine",evaluation_strategy="epoch")

A continuación definimos nuestro modelo, para ello usamos la clase `AutoModelForSequenceClassification` y vamos a utilizar un modelo pre-entrenado (recordar lo que era el transfer learning). Para ello solo tenemos que indicar el nombre de nuestro modelo (definido previamente en la variable `model_checkpoint` y el número de posibles valores que puede tomar nuestro clasificador (en este caso 5).  

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at mrm8488/electricidad-base-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ahora definimos la función que usaremos para calcular la precisión de nuestro modelo. En este caso usaremos la accuracy.

In [15]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

Ya podemos definir nuestro objeto `trainer` que usaremos para entrenar nuestro modelo. La estructura de este objeto será siempre la misma. Le tenemos que proporcionar:
1. El modelo.
2. La configuración del entrenamiento.
3. El conjunto de entrenamiento.
4. El conjunto de test.
5. El objeto que prepara los datos.
6. El tokenizador.
7. La métrica.

In [16]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=new_dataset["train"],
    eval_dataset=new_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Y ahora entrenamos el modelo mediante el método `train`. Este proceso puede llevar unos minutos y entrenará el modelo por 3 épocas (es decir mostrará todos los datos al modelo 3 veces). Este valor se puede cambiar en [la configuración del entrenamiento](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments).  

In [17]:
trainer.train()

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.347965,0.406452
2,1.407200,1.328409,0.425806
3,0.975900,1.464938,0.424516


TrainOutput(global_step=1164, training_loss=1.1383807388777585, metrics={'train_runtime': 161.3201, 'train_samples_per_second': 57.594, 'train_steps_per_second': 7.215, 'total_flos': 308308169529570.0, 'train_loss': 1.1383807388777585, 'epoch': 3.0})

Hemos obtenido una accuracy de aproximadamente el 44%. Esto puede variar de ejecución en ejecución ya que el entrenamiento de los modelos siempre tiene un factor aleatorio.

## Compartiendo el modelo

Una vez que tenemos entrenado nuestro modelo, nos interesa compartirlo con el resto del mundo para que puedan usarlo y también compararlo con otros modelos.

Es por ello que vamos a subir nuestro modelo al hub de huggingface. Para ello tenemos que ejecutar el siguiente comando.

In [18]:
# Vamos a la carpeta donde se ha guardado nuestro modelo, es el valor que
# definimos previamente en el objeto TrainingArguments
%cd clasificador-muchocine
# Subimos el modelo indicando un mensaje de confirmación, y una etiqueta.
trainer.push_to_hub(commit_message="Training complete", tags="classification")

/content/clasificador-muchocine


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RafSarLop/clasificador-muchocine/commit/36b10ddf8111bc5555d8e2530f3d7358c6499e8d', commit_message='Training complete', commit_description='', oid='36b10ddf8111bc5555d8e2530f3d7358c6499e8d', pr_url=None, pr_revision=None, pr_num=None)

Al terminar de ejecutarse el comando anterior tendremos nuestro modelo disponible en https://huggingface.co/joheras/clasificador-muchocine (en la URL anterior deberás reemplazar joheras por tu nombre de usuario).

Si accedes al enlace anterior, verás que tienes tu modelo disponible y una [tarjeta de modelo (o *model card*)](https://huggingface.co/docs/hub/model-cards) con una breve descripción del mismo. Es conveniente que proporciones información adicional a la *model card* ya que la que se genera de forma automática es demasiado básica.

Además verás que en el enlace anterior tienes un pequeño widget que te permite hacer predicciones con tu modelo.

Finalmente, vamos a ver cómo usar nuestro modelo para hacer predicciones desde código (esto puede ser útil sí por ejemplo nos interesa procesar múltiples textos de manera secuencial).


## Usando el modelo

En este caso al ser un modelo que hemos entrenado nosotros mismos podríamos usar los ficheros locales, pero vamos a ver cómo usar el modelo que acabamos de subir al hub de HuggingFace.

Para ello usamos un `pipeline` al que le debemos indicar el nombre del modelo que queremos descargar.

In [19]:
from transformers import pipeline
classifier = pipeline('text-classification', model='joheras/clasificador-muchocine')

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/730k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Ahora podemos hacer predicciones con nuestro modelo que tomará valores de label_0 (1 estrella) a label_4 (5 estrellas).

In [20]:
classifier('Es una obra mestra. Brillante.')

[{'label': 'LABEL_3', 'score': 0.4971005618572235}]

In [21]:
classifier('Es una película muy buena.')

[{'label': 'LABEL_3', 'score': 0.7302730679512024}]

In [22]:
classifier('Una buena película, sin más.')

[{'label': 'LABEL_3', 'score': 0.7171180248260498}]

In [23]:
classifier('Esperaba mucho más.')

[{'label': 'LABEL_2', 'score': 0.7051416635513306}]

In [24]:
classifier('He tirado el dinero. Una basura. Vergonzoso.')

[{'label': 'LABEL_0', 'score': 0.5983389019966125}]

Como podemos ver con los ejemplos anteriores, a pesar de que la accuracy del modelo no era excesivamente alta, para las frases anteriores funciona casi siempre correctamente. Con esto hemos visto cómo entrenar un modelo, compartirlo con el mundo, y usarlo.